In [ ]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, VotingClassifier, ExtraTreesClassifier,BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn import svm
from sklearn.metrics import roc_auc_score as AUC
from sklearn.metrics import accuracy_score as accuracy
import xgboost as xgb
import numpy as np
from sklearn.metrics import confusion_matrix, mean_squared_error, classification_report
from sklearn.grid_search import GridSearchCV
from sklearn import cross_validation, metrics
import sys
import os

In [ ]:
train_file = 'C:\\Users\\jacob\Dropbox\\Documents\\Python\\numerai\\numerai_training_data.csv'
test_file =  'C:\\Users\\jacob\Dropbox\\Documents\\Python\\numerai\\numerai_tournament_data.csv'
output_file = 'C:\\Users\\jacob\Dropbox\\Documents\\Python\\numerai\\predictions.csv'

train = pd.read_csv( train_file )
test = pd.read_csv( test_file )

train.drop( 'validation', axis = 1 , inplace = True )

# encode the categorical variable as one-hot, drop the original column afterwards
assert( set( train.c1.unique()) == set( test.c1.unique()))

train_dummies = pd.get_dummies( train.c1 )
train_num = pd.concat(( train.drop( 'c1', axis = 1 ), train_dummies.astype( int )), axis = 1 )

test_dummies = pd.get_dummies( test.c1 )
test_num = pd.concat(( test.drop( 'c1', axis = 1 ), test_dummies.astype(int) ), axis = 1 )

In [ ]:
#Make classifiers
clf1 = AdaBoostClassifier(DecisionTreeClassifier(max_depth=1),
                        algorithm="SAMME.R",
                        n_estimators=500)
clf2 = RandomForestClassifier(n_estimators=500)
clf3 = ExtraTreesClassifier(n_estimators=500)
clf4 = svm.SVC()
clf5 = svm.SVC(kernel='rbf', gamma=2)
clf_XGB =  xgb.XGBClassifier()
clf_XGB2 = xgb.XGBClassifier(max_depth=2, n_estimators=100)
clf_XGB3 = xgb.XGBClassifier(max_depth=2, n_estimators=200)
clf_XGB4 = xgb.XGBClassifier(max_depth=2, n_estimators=400)


clf_XGB5 = xgb.XGBClassifier(max_depth=4, n_estimators=100)
clf_XGB6 = xgb.XGBClassifier(max_depth=4, n_estimators=200)
clf_XGB7 = xgb.XGBClassifier(max_depth=4, n_estimators=400)

clf_XGB8 = xgb.XGBClassifier(max_depth=6, n_estimators=100)
clf_XGB9 = xgb.XGBClassifier(max_depth=6, n_estimators=200)
clf_XGB10 = xgb.XGBClassifier(max_depth=6, n_estimators=400)

eclf1 = VotingClassifier(estimators=[('AdaBoost', clf1), ('RF',clf2), ('ETC',clf3),('SVM',clf4),
                                     ('SVM2',clf5),('XGB',clf_XGB),('XGB2',clf_XGB2),('XGB3',clf_XGB3),
                                     ('XGB4',clf_XGB4),('XGB5',clf_XGB5),('XGB6',clf_XGB6),('XGB7',clf_XGB7),
                                     ('XGB8',clf_XGB8),('XGB9',clf_XGB9),('XGB10',clf_XGB10)], voting='soft')

bagging = BaggingClassifier(xgb.XGBClassifier(),n_estimators=100, verbose=3)

In [ ]:
#scores = cross_validation.cross_val_score(eclf1, train_num.drop( 'target', axis = 1 ), train_num.target, scoring = 'roc_auc', cv = 3, verbose = 1, n_jobs=-1 )
scores = cross_validation.cross_val_score(bagging, train_num.drop( 'target', axis = 1 ), train_num.target, scoring = 'roc_auc', cv = 3, verbose = 1, n_jobs=-1 )

In [ ]:
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

In [ ]:
import winsound
winsound.Beep(300,2000)

In [ ]:
p = bagging.predict_proba( test_num.drop( 't_id', axis = 1 ))

# save

test_num['probability'] = p[:,1]
test_num.to_csv( output_file, columns = ( 't_id', 'probability' ), index = None )

